# **Extreme Learning Machine**
- Pseudoinverse
- Classification Problem
- Iris dataset

In [1]:
import numpy as np
import numpy.matlib as matlib

from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler

In [2]:
# import iris data
# dataset is also be an numpy array object.
iris = datasets.load_iris()
x_train = iris.data
y_train = iris.target

scaler = MinMaxScaler(feature_range=(0, 1))
x_train = scaler.fit_transform(x_train)
y_train_onehot = np.eye(np.max(y_train) + 1)[y_train]

# data description
data_size = np.size(x_train, axis=0)
data_feature_size = np.size(x_train, axis=1)
target_feature_size = np.size(y_train_onehot, axis=1)

# swap data
swap_idx = np.random.permutation(data_size)
x_train = x_train[swap_idx]
y_train = y_train[swap_idx]
y_train_onehot = y_train_onehot[swap_idx]

In [3]:
# Train-Test Set ELM
train_set_size_percent = 2/3
train_set_size = int(data_size * train_set_size_percent)
test_set_size = data_size - train_set_size

# Train set
x_train_set = x_train[:train_set_size, :]
y_train_set_onehot = y_train_onehot[:train_set_size, :]
y_train_set = y_train[:train_set_size]

# Test set
x_test_set = x_train[train_set_size:, :]
y_test_set_onehot = y_train_onehot[train_set_size:, :]
y_test_set = y_train[train_set_size:]

In [4]:
y_test_set

array([1, 0, 1, 0, 1, 2, 2, 0, 2, 2, 1, 0, 0, 0, 2, 1, 1, 2, 2, 0, 0, 1,
       0, 0, 2, 1, 2, 0, 1, 2, 1, 0, 1, 1, 0, 2, 2, 0, 2, 1, 1, 1, 2, 0,
       2, 1, 2, 1, 2, 1])

In [5]:
def mse(A, B):
  return (np.square(np.sum(A - B, axis=1))).mean(axis=0)

# Activation Function
def sigmoid(x):
  return 1/(1 + np.exp(-x))

In [6]:
hidden_node_size=8
# Initialization model & Training
# hidden layer weight
elm_hidden_w = wi = np.random.rand(data_feature_size, hidden_node_size)
elm_hidden_bias_w = np.random.rand(1, hidden_node_size)

hidden_layer = sigmoid(x_train_set @ elm_hidden_w + matlib.repmat(elm_hidden_bias_w, train_set_size, 1))

# output layer weight
elm_output_w = np.linalg.pinv(hidden_layer) @ y_train_set_onehot

elm_output_w

array([[ -52.86508817,  176.77003332, -129.64196772],
       [ -33.39213552,  317.84841972, -277.2990621 ],
       [ -15.65163735, -160.54537387,  166.76861825],
       [ -27.85074022,  105.86103284,  -74.9940242 ],
       [  38.43761235,  -53.20742012,   17.84721373],
       [ -20.63115092,   25.35364721,   -7.49870508],
       [ 145.86390127, -535.03288294,  393.88881775],
       [ -34.04999517,  122.31411293,  -87.42340112]])

In [7]:
# Evaluation model - Train set
hidden_layer = sigmoid(x_train_set @ elm_hidden_w + matlib.repmat(elm_hidden_bias_w, train_set_size, 1))
output_layer = np.dot(hidden_layer , elm_output_w)

predict = np.argmax(output_layer, axis=1)
mse_val = mse(y_train_set_onehot, output_layer)

acc_val = (y_train_set == predict).sum() / train_set_size

print('Accuracy Train Set: {}'.format(acc_val))
print('MSE Train Set: {}'.format(mse_val))

Accuracy Train Set: 0.96
MSE Train Set: 4.341384658255754e-06
y_train_set: [1 0 2 2 0 1 0 2 1 0 1 1 2 1 2 1 1 0 1 0 1 0 2 2 2 1 2 2 0 0 0 2 1 1 1 0 0
 1 1 1 0 2 2 2 0 0 2 0 2 1 2 2 2 0 2 0 0 0 0 0 1 2 2 2 0 2 1 0 2 0 0 2 0 1
 1 2 2 1 1 2 0 1 2 2 1 2 1 2 0 0 1 0 1 1 0 0 0 0 1 1]
predict: [1 0 2 2 0 1 0 2 1 0 1 1 2 1 2 1 1 0 1 0 1 0 2 2 1 1 2 2 0 0 0 2 1 1 1 0 0
 1 1 1 0 2 2 2 0 0 2 0 1 1 2 2 2 0 2 0 0 0 0 0 1 2 1 2 0 2 1 0 2 0 0 2 0 1
 1 2 2 1 1 2 0 1 2 2 1 2 1 2 0 0 1 0 1 2 0 0 0 0 1 1]


In [10]:
# Evaluation model - Test set
hidden_layer = sigmoid(x_test_set @ elm_hidden_w + matlib.repmat(elm_hidden_bias_w, test_set_size, 1))
output_layer = np.dot(hidden_layer , elm_output_w)

predict = np.argmax(output_layer, axis=1)
mse_val = mse(y_test_set_onehot, output_layer)

acc_val = (y_test_set == predict).sum() / test_set_size

print('Accuracy Test Set: {}'.format(acc_val))
print('MSE Test Set: {}'.format(mse_val))

Accuracy Test Set: 0.94
MSE Test Set: 3.4264205995090073e-06
